In [1]:
import pandas as pd
import numpy as npn
from rouge import Rouge
import rouge
from torch.utils.data import DataLoader
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
import tensorflow as tf


c:\Users\JDari\Desktop\STT\StripTheText\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.


In [3]:
# Hier die größe der Test und Trainigsdaten Anpassen - auf dem Laptop geht das so nicht

train_data = load_dataset('xsum', split='train[:5]')#50000
print("Train Set: ",len(train_data))

test_data= load_dataset('xsum', split='test[:200]')
print("Test Set: ",len(test_data))

train_article = train_data['document'][0]
train_summary = train_data['summary'][0]
test_article = test_data['document'][0]
test_summary = test_data['summary'][0]

Found cached dataset xsum (C:/Users/JDari/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


Train Set:  5


Found cached dataset xsum (C:/Users/JDari/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


Test Set:  200


In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=50000)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

c:\Users\JDari\Desktop\STT\StripTheText\.venv\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JDari\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
def preprocess_function(examples):
    inputs = [doc for doc in examples['document']]
    targets = [summ for summ in examples['summary']]
    
    inputs_encoded = tokenizer.batch_encode_plus(inputs, truncation=True, padding='max_length', max_length=512)
    targets_encoded = tokenizer.batch_encode_plus(targets, truncation=True, padding='max_length', max_length=512)
    
    input_ids = torch.tensor(inputs_encoded['input_ids'])
    attention_mask = torch.tensor(inputs_encoded['attention_mask'])
    labels = torch.tensor(targets_encoded['input_ids'])
    
    return {'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels}
train_data_processed = train_data.map(preprocess_function, batched=True)
test_data_processed = test_data.map(preprocess_function, batched=True)

learning_rate = 1e-4
weight_decay = 0.01
num_train_epochs = 5
batch_size = 2

optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

#DataLoader
train_dataloader = DataLoader(train_data_processed, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data_processed, batch_size=batch_size, shuffle=True)

#Traininge
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

c:\Users\JDari\Desktop\STT\StripTheText\.venv\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [6]:
for epoch in range(num_train_epochs):
    model.train()
    total_loss = 0
    
    for batch in train_dataloader:
        input_ids = torch.cat(batch['input_ids'], dim=0)
        attention_mask = torch.cat(batch['attention_mask'], dim=0)
        labels = torch.cat(batch['labels'], dim=0)

        input_ids = input_ids.unsqueeze(0)
        attention_mask = attention_mask.unsqueeze(0)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=input_ids, labels=labels)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}: Average Loss = {average_loss:.4f}")
 

Epoch 1: Average Loss = 29.1445
Epoch 2: Average Loss = 21.0971
Epoch 3: Average Loss = 17.1258
Epoch 4: Average Loss = 13.1230
Epoch 5: Average Loss = 9.7285


In [7]:
# Evaluationsschleife
model.eval()
total_loss = 0

rouge_scorer = Rouge()

total_rouge1 = 0.0
total_rouge2 = 0.0
total_rougeL = 0.0


for batch in test_dataloader:
    input_ids = torch.cat(batch['input_ids'], dim=0)
    attention_mask = torch.cat(batch['attention_mask'], dim=0)
    labels = torch.cat(batch['labels'], dim=0)

    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    labels = labels.unsqueeze(0)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        loss = outputs.loss

        predicted_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
        predicted_sentences = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        reference_sentences = tokenizer.batch_decode(labels, skip_special_tokens=True)

        rouge_scores = rouge_scorer.get_scores(predicted_sentences, reference_sentences)
        total_rouge1 += rouge_scores[0]['rouge-1']['f']
        total_rouge2 += rouge_scores[0]['rouge-2']['f']
        total_rougeL += rouge_scores[0]['rouge-l']['f']
        
    total_loss += loss.item()

average_loss = total_loss / len(test_dataloader)
average_rouge1 = total_rouge1 / len(test_dataloader)
average_rouge2 = total_rouge2 / len(test_dataloader)
average_rougeL = total_rougeL / len(test_dataloader)
print(f"Test Loss: {average_loss:.4f}")
print(f"Average ROUGE-1: {average_rouge1:.4f}")
print(f"Average ROUGE-2: {average_rouge2:.4f}")
print(f"Average ROUGE-L: {average_rougeL:.4f}")

c:\Users\JDari\Desktop\STT\StripTheText\.venv\Lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Test Loss: 7.1850
Average ROUGE-1: 0.0279
Average ROUGE-2: 0.0008
Average ROUGE-L: 0.0261


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=50000)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [3]:
#speichern des modells 
modeloutput_dir = 'model/'
tokenizeroutput_dir = "tokenizer/"
model.save_pretrained(modeloutput_dir)
tokenizer.save_pretrained(tokenizeroutput_dir)

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json')

In [4]:
#laden des modells
loadedmodel = T5ForConditionalGeneration.from_pretrained(modeloutput_dir)
loadedtokenizer= T5Tokenizer.from_pretrained(tokenizeroutput_dir)

In [7]:
# Beispieltext
example_text = "This is an example input for summarization. It will be generated with nlp methodes like tokenization."
input_ids = loadedtokenizer.encode(example_text, truncation=True, padding='longest', return_tensors='pt')
output = loadedmodel.generate(input_ids=input_ids, max_length=100, num_beams=4, early_stopping=True)
output_text = loadedtokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Summary:", output_text)

Generated Summary: This example input will be generated with nlp methods like tokenization.


In [9]:
def create_spec_summ(input, rel_min):#input text als STR, rel_min ist der Prozentualteil auf den reduziert werden soll.(reduktion die mindestens erreicht werden soll)
    summ_ready=False
    counter=0
    input_help=input
    while counter<20 and summ_ready==False:
        input_ids = loadedtokenizer.encode(input_help, truncation=True, padding='longest', return_tensors='pt')
        output_pre = loadedmodel.generate(input_ids=input_ids, max_length=100, num_beams=4, early_stopping=True)
        output_text = loadedtokenizer.decode(output_pre[0], skip_special_tokens=True)
        if len(output_text)/len(input)<rel_min:
            summ_ready=True
        else:
            input_help=output_text
    rouge_scorer = Rouge()
    rouge_scores = rouge_scorer.get_scores(output_text, input)

    # Ausgabe der ROUGE-Werte
    print("ROUGE-1 Score:", rouge_scores[0]['rouge-1']['f'])
    print("ROUGE-2 Score:", rouge_scores[0]['rouge-2']['f'])
    print("ROUGE-L Score:", rouge_scores[0]['rouge-l']['f'])
    if counter==20:
        return "!!!Gewünschte Kompression konnte nicht erreicht werden!!!"+output_text
    else:
        return output_text

In [11]:
test="John Mowbray, 3rd Duke of Norfolk (1415–1461), was a fifteenth-century English magnate who was the only son and heir of John Mowbray, 2nd Duke of Norfolk, and Katherine Neville. As a minor he became a ward of King Henry VI and was placed under the protection of Humphrey, Duke of Gloucester, alongside whom Mowbray would later campaign in France. He led the defence of England's possessions in Normandy during the Hundred Years' War. He fought in Calais in 1436, and in 1437 and 1438 served as warden of the Eastern March on the Anglo-Scottish border. In the early 1430s he became the bitter rival of William de la Pole, Earl (later Duke) of Suffolk. In the early years of the Wars of the Roses in the 1450s he defended King Henry against two rebellions by Richard, Duke of York."
dest_rel=0.8
print(create_spec_summ(test,dest_rel))

3rd Duke of Norfolk (1415–1461), was a 15th-century English magnate who was the only son and heir of John Mowbray, 2nd Duke of Norfolk, and Katherine Neville. Mowbray was the son of John Mowbray, 2nd Duke of Norfolk, and Katherine Neville.
